In [1]:
from typing import Tuple, Any

import dice_ml
from dice_ml import Dice
from dice_ml.utils.exception import UserConfigValidationException

from sklearn.datasets import load_iris, fetch_california_housing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from src.transition_system import transition_system, indexs_for_window, list_to_str

import pandas as pd
import os
import random
import utils
from math import ceil
from timeout_decorator import timeout, TimeoutError
from collections import Counter
%load_ext autoreload
%autoreload 2

# Dataset: VINST (Volvo Belgium)
link: https://www.win.tue.nl/bpi/doku.php?id=2013:challenge

## Setup Variables

In [2]:
pd.options.display.max_columns= None
SECONDS_TO_HOURS = 60 * 60
SECONDS_TO_DAYS = 60 * 60 * 24
case_id_name = 'SR_Number'  # The case identifier column name.
start_date_name = 'Change_Date+Time'  # Maybe change to start_et (start even time)
activity_column_name = "ACTIVITY"

## Load Data

In [3]:
data_dir = "../preprocessed_datasets/"
train_dataset_file = "train-set-cfe.csv"
test_dataset_file = "test-set-cfe.csv"
df = pd.read_csv("../data/VINST cases incidents.csv")  # Use full dataset for transition systens
df_train = pd.read_csv(os.path.join(data_dir, train_dataset_file))
df_test = pd.read_csv(os.path.join(data_dir, test_dataset_file))

### Basic Preprocessing of the dataset

In [4]:
df = df.fillna("missing")
df_train = df_train[:20_000]  # 31_289
# df_test = df_test[: 19_041]
print(f"Rows in df_train: {len(df_train)}")
# df_train.info()

Rows in df_train: 20000


### Helper functions

In [17]:
def get_case_id(df, case_id_name=case_id_name, multi=False):
    return df[case_id_name].unique().item()

def get_query_instance(sidx=14, eidx=16):
    assert eidx - sidx == 2, "One row represents the current action and the next one represents the suggested action"
    current_step = X_train[sidx: sidx+1]
    expected_next_step = X_train[eidx-1: eidx]
    return current_step, expected_next_step

def activity_n_resources(df, resources_columns=None, threshold_percentage=100):
    """
    Creates a set of tuples, each tuple has elements from the columns specified through `resource_columns`.
    E.g. { ('action_1', 'rresource_1'), ... (activity_3, resource_5) }
    Args:
        df (pd.DataFrame):
        resources_columns (list): columns that contains the activity and resources.
    Returns:
        Set of tuples. A single element contains the activity and resources of a single row from the
        dataframe.
    """
    if resources_columns is None:
        # raise TypeError("Please specify the columns that have resources")
        resources_columns = [activity_column_name, 'Involved_ST_Function_Div', 'Involved_Org_line_3',
                             'Involved_ST', 'Country', 'Owner_Country']

    threshold = threshold_percentage / 100

    valid_activity_n_resource = set( df[resources_columns].apply(tuple, axis='columns') )

    # combo: combination
    resource_combo_frequency = {}

    valid_resource_combo = df[resources_columns].apply(tuple, axis='columns')

    for elem in valid_resource_combo:
        if resource_combo_frequency.get(elem):
            resource_combo_frequency[elem] += 1
        else:
            resource_combo_frequency[elem] = 1
    # Creates a list of (combo, counts)
    resource_combo_counts = [ (k, v) for k, v in resource_combo_frequency.items() ]
    sorted_resource_combo_counts = sorted( resource_combo_counts, key=lambda item: item[1], reverse=True )
    sorted_combos = [combo for combo, _ in sorted_resource_combo_counts ]
    amount_of_combos_to_select = int( len(sorted_combos) * threshold ) + 1
    valid_activity_n_resources = sorted_combos[:amount_of_combos_to_select]
    return valid_activity_n_resources

# current_step, query_instances = get_query_instance(14, 16)

# valid_resources = activity_n_resources(df_train, ["ACTIVITY", "Involved_ST_Function_Div", "Involved_Org_line_3", "Involved_ST"])
resource_columns_to_validate = [activity_column_name, 'Involved_ST_Function_Div', 'Involved_Org_line_3',
                                'Involved_ST', 'Country', 'Owner_Country']
valid_resources = activity_n_resources(df, resource_columns_to_validate)
# len(valid_resources)

In [6]:
# threshold = 0.8
# resource_combo_frequency = {}
# resource_column_names = [activity_column_name, 'Involved_ST_Function_Div', 'Involved_Org_line_3',
#                          'Involved_ST', 'Country', 'Owner_Country']
# valid_resource_combo = df[resource_column_names].apply(tuple, axis='columns')
#
# for elem in valid_resource_combo:
#     if resource_combo_frequency.get(elem):
#         resource_combo_frequency[elem] += 1
#     else:
#         resource_combo_frequency[elem] = 1
# # Creates a list of (combo, counts)
# resource_combo_counts = [ (k, v) for k, v in resource_combo_frequency.items() ]
# sorted_resource_combo_counts = sorted( resource_combo_counts, key=lambda item: item[1], reverse=True )
# sorted_combos = [combo for combo, _ in sorted_resource_combo_counts ]
# amount_of_combos_to_select = int( len(sorted_combos) * threshold ) + 1
# high_frequency_combos = sorted_combos[:amount_of_combos_to_select]

### Prepare the Test Dataset

In [7]:
def get_test(df, case_id_name):
    # df_result = pd.DataFrame(columns=df.columns)
    result_lst = []

    for idx in df[case_id_name].unique():
        df_trace = df[df[case_id_name] == idx]
        # ceil enables cases with 1 row to pass through
        cut = ceil(len(df_trace) * random.uniform(0.5, 0.7)) #+ 2  # 2 because one for the floor and one for the pred
        df_trace = df_trace.iloc[:cut].reset_index(drop=True)

        # df_result = pd.concat([df_result, df_trace])
        result_lst.append(df_trace.reset_index(drop=True))
        # break
    # return df_result.reset_index(drop=True)
    return result_lst

test_cases = get_test(df_test, case_id_name)

In [8]:
# ### Features that can vary
# option 1:
# cols_to_vary = [col for col in df_train.columns if col[0] == '#']
# cols_to_vary.extend(["ACTIVITY"])

# option 2:
cols_to_vary = ["ACTIVITY", "Involved_ST_Function_Div", "Involved_Org_line_3", "Involved_ST"]

outcome_name = "lead_time"
def prepare_df_for_ml(df, outcome_name, columns_to_remove=None) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    :param str outcome_name: name of the target column.
    """
    # Before training for ml we need to remove columns that can are not needed for ML model.
    if columns_to_remove is None:
        columns_to_remove = ["Change_Date+Time", "time_remaining"]
    df = df.drop([case_id_name], axis="columns")
    df = df.drop(columns_to_remove, axis="columns")
    X = df.drop([outcome_name], axis=1)
    y = df[outcome_name]
    return X, y

X_train, y_train = prepare_df_for_ml(df_train, outcome_name)
# X_test, y_test = prepare_df_for_ml(df_test, outcome_name)

continuous_features = ["time_from_first", "time_from_previous_et", "time_from_midnight", "# ACTIVITY=In Progress", "# ACTIVITY=Awaiting Assignment",
                       "# ACTIVITY=Resolved", "# ACTIVITY=Assigned", "# ACTIVITY=Closed", "# ACTIVITY=Wait - User", "# ACTIVITY=Wait - Implementation", "# ACTIVITY=Wait",
                       "# ACTIVITY=Wait - Vendor", "# ACTIVITY=In Call", "# ACTIVITY=Wait - Customer", "# ACTIVITY=Unmatched", "# ACTIVITY=Cancelled"]
categorical_features = ["Status", "ACTIVITY", "Involved_ST_Function_Div", "Involved_Org_line_3", "Involved_ST", "SR_Latest_Impact", "Product", "Country", "Owner_Country",
                        "weekday"]

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                           ('classifier', RandomForestRegressor(n_jobs=7))])
model = clf.fit(X_train, y_train)

# ## Create DiCE model
d_iris = dice_ml.Data(dataframe=pd.concat([X_train, y_train], axis="columns"),
                      continuous_features=continuous_features,
                      outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
m_iris = dice_ml.Model(model=model, backend="sklearn", model_type='regressor')
method = "random"  # genetic, kdtree, random
exp_genetic_iris = Dice(d_iris, m_iris, method=method)  # Method random does not support features_weights argument in generate_counterfactuals()
# exp_genetic_iris = Dice(d_iris, m_iris, method="kdtree")

## Experiment with Single Queries

In [53]:
sidx = 60
eidx = 110

df_train[sidx: eidx]
# df_train[sidx: eidx]

,SR_Number,Change_Date+Time,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,time_remaining,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
60,1-504538555,1299000865,Accepted,In Progress,A2_1,Org line C,D2,Medium,PROD706,nl,Belgium,102052.0,9249.0,63265.0,Tuesday,37777625.0,3,2,0,0,0,0,0,0,0,0,0,0,0,37879677.0
61,1-504538555,1299001560,Accepted,Assigned,A2_1,Org line C,D2,Medium,PROD706,nl,Belgium,102747.0,695.0,63960.0,Tuesday,37776930.0,4,2,0,0,0,0,0,0,0,0,0,0,0,37879677.0
62,1-504538555,1299048169,Accepted,In Progress,A2_1,Org line C,D2,Medium,PROD706,nl,Belgium,149356.0,46609.0,24169.0,Wednesday,37730321.0,4,2,0,1,0,0,0,0,0,0,0,0,0,37879677.0
63,1-504538555,1299048251,Queued,Awaiting Assignment,missing,Org line V7n,V32 2nd,Medium,PROD706,nl,Belgium,149438.0,82.0,24251.0,Wednesday,37730239.0,5,2,0,1,0,0,0,0,0,0,0,0,0,37879677.0
64,1-504538555,1304525335,Accepted,In Progress,missing,Org line V7n,V32 2nd,Medium,PROD706,nl,Netherlands,5626522.0,5477084.0,58135.0,Wednesday,32253155.0,5,3,0,1,0,0,0,0,0,0,0,0,0,37879677.0
65,1-504538555,1331632423,Queued,Awaiting Assignment,missing,Org line V7n,V37 2nd,Medium,PROD706,nl,Netherlands,32733610.0,27107088.0,35623.0,Tuesday,5146067.0,6,3,0,1,0,0,0,0,0,0,0,0,0,37879677.0
66,1-504538555,1331633484,Accepted,In Progress,missing,Org line V7n,V37 2nd,Medium,PROD706,nl,Netherlands,32734671.0,1061.0,36684.0,Tuesday,5145006.0,6,4,0,1,0,0,0,0,0,0,0,0,0,37879677.0
67,1-504538555,1331634440,Queued,Awaiting Assignment,missing,Org line V7n,V32 2nd,Medium,PROD706,nl,Netherlands,32735627.0,956.0,37640.0,Tuesday,5144050.0,7,4,0,1,0,0,0,0,0,0,0,0,0,37879677.0
68,1-504538555,1333461567,Accepted,In Progress,missing,Org line V7n,V32 2nd,Medium,PROD706,nl,Netherlands,34562754.0,1827127.0,50367.0,Tuesday,3316923.0,7,5,0,1,0,0,0,0,0,0,0,0,0,37879677.0
69,1-504538555,1333547611,Accepted,Wait - Implementation,missing,Org line V7n,V32 2nd,Medium,PROD706,nl,Netherlands,34648798.0,86044.0,50011.0,Wednesday,3230879.0,8,5,0,1,0,0,0,0,0,0,0,0,0,37879677.0


In [58]:
sidx = 94
eidx = 95
total_time_upper_bound = int( y_train[sidx] * (90 / 100) )  # A percentage of the original total time of the trace
query_instances = X_train[sidx: eidx]
if method == "genetic":  # or method == "kdtree":
    feature_weights = {"ACTIVITY": 1.0, "Involved_ST_Function_Div": 1.0, "Involved_Org_line_3": 1.0, "Involved_ST": 1.0}
    # feature_weights = {"ACTIVITY": 1, "Involved_ST_Function_Div": 1}
    cfe = exp_genetic_iris.generate_counterfactuals(query_instances, total_CFs=15, desired_range=[0, total_time_upper_bound], features_to_vary=cols_to_vary,
                                                    feature_weights=feature_weights)
else:
    cfe = exp_genetic_iris.generate_counterfactuals(query_instances, total_CFs=200, desired_range=[0, total_time_upper_bound], features_to_vary=cols_to_vary)

cfe.visualize_as_dataframe(show_only_changes=True)

100%|██████████| 1/1 [00:07<00:00,  7.28s/it]

Query instance (original outcome : 32735702)


,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,Queued,Awaiting Assignment,A2_1,Org line A2,D8,Medium,PROD542,nl,Netherlands,195784.0,91.0,57912.0,Wednesday,12,7,0,0,0,2,0,0,0,0,0,0,0,32735702.0



Diverse Counterfactual set (new outcome: [0, 33558805])


,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,-,-,-,-,V12 3rd,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,23627424.0
1,-,Unmatched,E_6,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,-,Wait - Customer,-,-,G179,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,23814848.0
3,-,-,-,-,D9,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,23627424.0
4,-,Unmatched,-,Org line V7,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,31255762.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,-,In Call,E_4,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
196,-,Closed,-,Org line G1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,31255762.0
197,-,-,-,-,U2,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,23627424.0
198,-,-,-,-,G142 2nd,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,23627424.0


## Apply Transition System Validation to the above Single Query Results

In [25]:
window_size = 3
activity_n_resources_columns = ["ACTIVITY", "Involved_ST_Function_Div"]

# === Load the Transition Graph
_, transition_graph = transition_system(df, case_id_name=case_id_name, activity_column_name=activity_column_name, window_size=window_size)

In [60]:
def validate_transition(cfe, transition_graph=transition_graph, window_size=window_size):
    """  resource_columns_to_validate=None possible future parameter
    Args:
        cfe (dice_ml.counterfactual_explanations.CounterfactualExplanations): Dice counterfactual explanations object.
    """
    # if resource_columns_to_validate is None:
    #     activity_n_resources_columns = ["ACTIVITY", "Involved_ST_Function_Div"]
    cf_examples_df = cfe.cf_examples_list[0].final_cfs_df.copy()  # Get the counterfactual explanations dataframe from the object

    # === Get the prefix of trace for which to validate the next activity
    query_case_id = get_case_id( df_train[sidx: eidx] )
    df_query = df_train[ df_train[case_id_name] == query_case_id ]

    index_to_previous_activity = sidx - 1

    start_index, end_index = indexs_for_window(index_to_previous_activity, window_size=window_size, end_exclusive=False)
    prefix_of_activities = df_query.loc[start_index: end_index, activity_column_name].to_list()  # loc is used to access the index values inside the dataframe
    prefix_of_activities = list_to_str(prefix_of_activities)

    # === Verify the next activity
    indexes_to_drop = []
    for i, suggested_next_activity in cf_examples_df[activity_column_name].items():
        if suggested_next_activity not in transition_graph[prefix_of_activities]:
            indexes_to_drop.append(i)
            print(i, suggested_next_activity)

    cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)

    # === Verify the associated resources
    indexes_to_drop = []
    for i, row in cf_examples_df[ resource_columns_to_validate ].iterrows():
        row_tuple = tuple(row)
        if row_tuple not in valid_resources:
            # print(f"removed row had: {row_tuple}")
            indexes_to_drop.append(i)

    cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)
    return cf_examples_df

# cf_examples_df = cfe.cf_examples_list[0].final_cfs_df.copy()
# # === Get the prefix of trace for which to validate the next activity
# query_case_id = get_case_id( df_train[sidx: eidx] )
# df_query = df_train[ df_train[case_id_name] == query_case_id ]
#
# index_to_previous_activity = sidx - 1
#
# start_index, end_index = indexs_for_window(index_to_previous_activity, window_size=window_size, end_exclusive=False)
# prefix_of_activities = df_query.loc[start_index: end_index, activity_column_name].to_list()  # loc is used to access the index values inside the dataframe
# prefix_of_activities = list_to_str(prefix_of_activities)
#
# # === Verify the next activity
# indexes_to_drop = []
# for i, suggested_next_activity in cf_examples_df[activity_column_name].items():
#     if suggested_next_activity not in transition_graph[prefix_of_activities]:
#         indexes_to_drop.append(i)
#         # print(i, suggested_next_activity)
#
# cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)
# print("Length of cf_examples_df", len(cf_examples_df))
#
# # === Verify the associated resources
# indexes_to_drop = []
# for idx, row in cf_examples_df[ resource_columns_to_validate ].iterrows():
#     row_tuple = tuple(row)
#     print("row tip",row_tuple)
#     if row_tuple not in valid_resources:
#         # print(f"removed row had: {row_tuple}")
#         indexes_to_drop.append(idx)
#
# cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)
# cf_examples_df
# # cf_examples_df = validate_transition(cfe, resource_columns_to_validate)

In [61]:
# list( filter(lambda x: x[4] == "nl" and x[5] == "Netherlands", valid_resources) )

In [230]:
cfe.visualize_as_dataframe(show_only_changes=True)
print(f"Valid Counterfactual Set")
cf_examples_df

Query instance (original outcome : 41260204)


,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,Accepted,Assigned,E_10,Org line C,G140 2nd,Medium,PROD453,se,Sweden,3617038.0,321053.0,24980.0,Monday,7,2,0,1,0,2,2,1,0,0,0,0,0,41260204.0



Diverse Counterfactual set (new outcome: [0, 37134182])


,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,-,-,V3_3,Org line G1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0
1,-,-,E_6,Org line G1,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0
2,-,-,E_7,Org line V10,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0
3,-,-,-,Org line H,V50 2nd,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0
4,-,-,A2_4,Org line V11,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0
5,-,Wait,-,Org line V2,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0
6,-,-,A2_1,Org line V9,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,32859970.0


,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,Accepted,Assigned,E_7,Org line V10,G140 2nd,Medium,PROD453,se,Sweden,3617038.0,321053.0,24980.0,Monday,7,2,0,1,0,2,2,1,0,0,0,0,0,32859970.0
1,Accepted,Assigned,E_10,Org line H,V50 2nd,Medium,PROD453,se,Sweden,3617038.0,321053.0,24980.0,Monday,7,2,0,1,0,2,2,1,0,0,0,0,0,32859970.0
2,Accepted,Assigned,A2_4,Org line V11,G140 2nd,Medium,PROD453,se,Sweden,3617038.0,321053.0,24980.0,Monday,7,2,0,1,0,2,2,1,0,0,0,0,0,32859970.0
3,Accepted,Wait,E_10,Org line V2,G140 2nd,Medium,PROD453,se,Sweden,3617038.0,321053.0,24980.0,Monday,7,2,0,1,0,2,2,1,0,0,0,0,0,32859970.0
4,Accepted,Assigned,A2_1,Org line V9,G140 2nd,Medium,PROD453,se,Sweden,3617038.0,321053.0,24980.0,Monday,7,2,0,1,0,2,2,1,0,0,0,0,0,32859970.0


### Check Feature Importance score

In [47]:
imp = exp_genetic_iris.local_feature_importance(query_instances, total_CFs=10, desired_range=[0, total_time_upper_bound], features_to_vary=cols_to_vary)
imp.local_importance

100%|██████████| 1/1 [00:00<00:00,  1.63it/s]


[{'Involved_ST': 1.0,
  'ACTIVITY': 0.3,
  'Involved_ST_Function_Div': 0.3,
  'Involved_Org_line_3': 0.1,
  'Status': 0.0,
  'SR_Latest_Impact': 0.0,
  'Product': 0.0,
  'Country': 0.0,
  'Owner_Country': 0.0,
  'weekday': 0.0,
  'time_from_first': 0.0,
  'time_from_previous_et': 0.0,
  'time_from_midnight': 0.0,
  '# ACTIVITY=In Progress': 0.0,
  '# ACTIVITY=Awaiting Assignment': 0.0,
  '# ACTIVITY=Resolved': 0.0,
  '# ACTIVITY=Assigned': 0.0,
  '# ACTIVITY=Closed': 0.0,
  '# ACTIVITY=Wait - User': 0.0,
  '# ACTIVITY=Wait - Implementation': 0.0,
  '# ACTIVITY=Wait': 0.0,
  '# ACTIVITY=Wait - Vendor': 0.0,
  '# ACTIVITY=In Call': 0.0,
  '# ACTIVITY=Wait - Customer': 0.0,
  '# ACTIVITY=Unmatched': 0.0,
  '# ACTIVITY=Cancelled': 0.0}]

## Experiment with Multiple Queries

In [17]:
test_cases[i+1]

,SR_Number,Change_Date+Time,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,time_remaining,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,1-643733486,1326813444,Accepted,In Progress,A2_1,Org line C,D6,Medium,PROD542,nl,Belgium,0.0,0.0,55044.0,Tuesday,9965027.0,0,0,0,0,0,0,0,0,0,0,0,0,0,9965027.0
1,1-643733486,1326813477,Accepted,In Progress,A2_1,Org line C,D6,Medium,PROD542,nl,Belgium,33.0,33.0,55077.0,Tuesday,9964994.0,1,0,0,0,0,0,0,0,0,0,0,0,0,9965027.0
2,1-643733486,1326813558,Queued,Awaiting Assignment,missing,Org line V7n,V37 2nd,Medium,PROD542,nl,Belgium,114.0,81.0,55158.0,Tuesday,9964913.0,2,0,0,0,0,0,0,0,0,0,0,0,0,9965027.0
3,1-643733486,1326977851,Accepted,In Progress,missing,Org line V7n,V37 2nd,Medium,PROD542,nl,Netherlands,164407.0,164293.0,46651.0,Thursday,9800620.0,2,1,0,0,0,0,0,0,0,0,0,0,0,9965027.0
4,1-643733486,1326977939,Queued,Awaiting Assignment,A2_1,Org line C,D6,Medium,PROD542,nl,Netherlands,164495.0,88.0,46739.0,Thursday,9800532.0,3,1,0,0,0,0,0,0,0,0,0,0,0,9965027.0
5,1-643733486,1326982060,Accepted,In Progress,A2_1,Org line C,D6,Medium,PROD542,nl,Belgium,168616.0,4121.0,50860.0,Thursday,9796411.0,3,2,0,0,0,0,0,0,0,0,0,0,0,9965027.0
6,1-643733486,1326982072,Accepted,Assigned,A2_1,Org line C,D6,Medium,PROD542,nl,Belgium,168628.0,12.0,50872.0,Thursday,9796399.0,4,2,0,0,0,0,0,0,0,0,0,0,0,9965027.0
7,1-643733486,1326983196,Accepted,In Progress,A2_1,Org line C,D6,Medium,PROD542,nl,Belgium,169752.0,1124.0,51996.0,Thursday,9795275.0,4,2,0,1,0,0,0,0,0,0,0,0,0,9965027.0


In [73]:
@timeout(120)  # Set a timeout of 5 seconds
def generate_cfe(query_instances, total_time_upper_bound):
    if method == "genetic":
        cfe = exp_genetic_iris.generate_counterfactuals(query_instances, total_CFs=10, desired_range=[0, total_time_upper_bound], features_to_vary=cols_to_vary,
                                                        feature_weights = {"ACTIVITY": 1, "Involved_ST_Function_Div": 1, "Involved_Org_line_3": 1, "Involved_ST": 1},
                                                        proximity_weight=proximity_weight, sparsity_weight=sparsity_weight, diversity_weight=diversity_weight)
    else:
        cfe = exp_genetic_iris.generate_counterfactuals(query_instances, total_CFs=200, desired_range=[0, total_time_upper_bound], features_to_vary=cols_to_vary,
                                                        proximity_weight=proximity_weight, sparsity_weight=sparsity_weight, diversity_weight=diversity_weight)
    return cfe

In [74]:
%%capture
i = 0
cfe_before_validation = []
cfe_after_validation = []
cfe_not_found = []
cases_includes_new_data = []
cases_too_small = []
for df_test_trace in test_cases:
    i+= 1
    if i == 20:
        break

    if 0 < len(df_test_trace) <= 2:
        print("too small", i, df_test_trace[case_id_name].unique().item())
        cases_too_small.append( get_case_id(df_test_trace, multi=True) )
        continue

    X_test, y_test = prepare_df_for_ml(df_test_trace, outcome_name)
    # Access the last row of the truncated trace to replicate the behavior of a running trace
    total_time_upper_bound = int( y_test.iloc[-1] * (90 / 100) )  # A percentage of the original total time of the trace
    query_instances = X_test.iloc[-1:]
    proximity_weight = 0.2
    sparsity_weight = 0.2
    diversity_weight = 5.0
    try:
        cfe = generate_cfe(query_instances, total_time_upper_bound)
        cfe_before_validation.append( (get_case_id(df_test_trace), cfe) )
        cfe = validate_transition(cfe)
        if len(cfe) > 0:
            cfe_after_validation.append( (get_case_id(df_test_trace), cfe) )

    except UserConfigValidationException:
        cfe_not_found.append(get_case_id(df_test_trace))
    except TimeoutError:  # When function takes too long
        cfe_not_found.append(get_case_id(df_test_trace))
    except ValueError:
        # print(f"Includes feature not found in training data: {get_case_id(df_test_trace)}")
        cases_includes_new_data.append(get_case_id(df_test_trace))



In [75]:
data = { "cfe_before_validation": [ len(cfe_before_validation) ],
         "cfe_after_validation": [ len(cfe_after_validation) ],
         "cfe_not_found": [ len(cfe_not_found) ],
         "cases_includes_new_data": [ len(cases_includes_new_data)],
         "cases_too_small": [ len(cases_too_small) ]}
df_result = pd.DataFrame(data)
df_result

,cfe_before_validation,cfe_after_validation,cfe_not_found,cases_includes_new_data,cases_too_small
0,16,12,1,2,0


In [387]:
# for _, cfe in cfes_list:
#     cfe.visualize_as_dataframe(show_only_changes=True)
cfes_list[9][1]

,Status,ACTIVITY,Involved_ST_Function_Div,Involved_Org_line_3,Involved_ST,SR_Latest_Impact,Product,Country,Owner_Country,time_from_first,time_from_previous_et,time_from_midnight,weekday,# ACTIVITY=In Progress,# ACTIVITY=Awaiting Assignment,# ACTIVITY=Resolved,# ACTIVITY=Assigned,# ACTIVITY=Closed,# ACTIVITY=Wait - User,# ACTIVITY=Wait - Implementation,# ACTIVITY=Wait,# ACTIVITY=Wait - Vendor,# ACTIVITY=In Call,# ACTIVITY=Wait - Customer,# ACTIVITY=Unmatched,# ACTIVITY=Cancelled,lead_time
0,Accepted,Resolved,V3_2,Org line C,G7 3rd,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,1188408.0
1,Accepted,In Progress,V3_2,Org line H,S45 2nd,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,1810092.0
2,Accepted,Wait,V3_2,Org line C,V33,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,1833501.0
3,Accepted,In Progress,V3_2,Org line V11,G49,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,2512714.0
4,Accepted,In Progress,V3_2,Org line C,L4 2nd,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,1681719.0
5,Accepted,In Progress,missing,Org line C,G49,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,1667908.0
6,Accepted,Wait - User,V3_2,Org line V8,G49,Medium,PROD562,se,Sweden,600134.0,401.0,45412.0,Monday,6,7,0,0,0,1,0,0,0,0,0,0,0,1819502.0


## Observation
The output usually comes within 2 minutes

#### TODOS
- Check how many queries (rows) have valid CFEs
- How to make CFE not, ACTIVITY column not to take the value "Pending Liquidation request"
- Discover how DiCE handle constraints ( For us do it in post-processing step )
- CE_OU - resource column, you can modify it
- Modify just the resource (col: CE_UO) & Activity (col: ACTIVITY)
- build a transition system to validate activity and then resource
- Do the above for total time case
- I
- Implement activity validation and resource validation to the above
-   To do this effectively maybe figure out how the DiCE post process the constraints and implement this in that layer

In [ ]:
query_instances = X_train[sidx: eidx]
cfe = exp_genetic_iris.generate_counterfactuals(query_instances, total_CFs=5, desired_class="opposite", features_to_vary=cols_to_vary)
cfe.visualize_as_dataframe(show_only_changes=True)

genetic_cfes = []

for idx, query_instances in X_train.iterrows():
    query_instances = query_instances.to_frame().transpose()
    # query_instances = X_train[0:1]  # an interesting query`

    cfe = exp_genetic_iris.generate_counterfactuals(query_instances, total_CFs=5, desired_class="opposite", features_to_vary=cols_to_vary)

    genetic_cfes.append( (idx, cfe) )

-Implement post processing steps
    - Transition transition_system with full traces
    - 4 tuple comparison with country and owner country



## Deprecated Code

In [ ]:
def validate_transition(cfe):
    """ Deprecated function.
    Works with transition graph which has single activity as key.
    Args:
        cfe: Dice counterfactual object.
    """
    cf_examples_df = cfe.cf_examples_list[0].final_cfs_df.copy()

    # expected_next_action = cfe.cf_examples_list[0].test_instance_df[activity_column_name].item()
    # print(f"Expected next action: {expected_next_action}")

    current_activity = current_step[activity_column_name].item()
    # === Verify the next activity
    indexes_to_drop = []
    for idx, suggested_next_activity in cf_examples_df[activity_column_name].items():
        # print(f"index: {idx}, Suggest: {suggested_next_activity}")
        if suggested_next_activity not in transition_graph[current_activity]:
            indexes_to_drop.append(idx)

    cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)

    # === Verify the associated resources
    indexes_to_drop = []
    for idx, row in cf_examples_df[ ["ACTIVITY", "Involved_ST_Function_Div"] ].iterrows():
        row_tuple = tuple(row)
        if row_tuple not in valid_resources:
            # print(f"removed row had: {row_tuple}")
            indexes_to_drop.append(idx)

    cf_examples_df = cf_examples_df.drop(indexes_to_drop, axis='index').reset_index(drop=True)
    return cf_examples_df